# Projeção Populacional

## Preparação do Notebook

Na célula adiante são feitos procedimentos de preparação do notebook para trabalho com os dados, tais como:
1. mapeamento do diretório em que se encontra o pacote br_demography;
2. importação de bibliotecas necessárias para processamento dos dados;
3. carregametno de project_id para faturamento no Google Cloud, conforme requerido pela biblioteca basedosdados.

In [18]:
# adding br_demography to Python Path
import sys 
sys.path.append("../../")

from br_demography  import municipality_migration as mm
from br_demography  import municipality_births as mb
from br_demography  import municipality_pop_pyramid as mep

import basedosdados as bd
from dotenv import load_dotenv
import os
import pandas as pd
import numpy as np
# Carrega as variáveis de ambiente do arquivo .env
load_dotenv(dotenv_path='../../.env')

# Use as variáveis de ambiente conforme necessário
project_id = os.getenv('GOOGLE_CLOUD_PROJECT_ID')

# Projeção para 2032

## População do Ano Inicial

In [19]:
categories = pd.CategoricalDtype(
    categories=[
        '0 a 9 anos','10 a 19 anos', '20 a 29 anos','30 a 39 anos'
        , '40 a 49 anos', '50 a 59 anos', '60 a 69 anos'
        , '70 a 79 anos','80 anos ou mais'
        ]
    ,ordered=True
)

df_pop = pd.read_csv(
    '../../br_demography/results/tab/pop_municipios_rmc_2000_2022_estimativa_intercensitaria.csv'
    , decimal=','
    , sep=';'
    , index_col=['Município', 'Sexo', 'Faixa Etária']
    , dtype={'Faixa Etária':categories}
    )
df_pop.columns = df_pop.columns.astype(int)
df_pop.sort_index(inplace=True)
df_pop.iloc[np.r_[0:2, -2:0]]

2000  2001  2002  2003  2004  2005  \
Município       Sexo      Faixa Etária                                          
Adrianópolis    Feminino  0 a 9 anos        704   681   660   639   619   599   
                          10 a 19 anos      823   798   775   752   730   708   
Tunas do Paraná Masculino 70 a 79 anos       60    60    61    62    63    64   
                          80 anos ou mais    11    11    12    13    14    15   

                                           2006  2007  2008  2009  ...  2013  \
Município       Sexo      Faixa Etária                             ...         
Adrianópolis    Feminino  0 a 9 anos        580   562   544   527  ...   485   
                          10 a 19 anos      687   667   647   628  ...   576   
Tunas do Paraná Masculino 70 a 79 anos       65    66    67    68  ...    76   
                          80 anos ou mais    16    17    18    19  ...    24   

                                           2014  2015  2016  2017  2018  2019  \
Município       Sexo      Faixa Etária                                          
Adrianópolis    Feminino  0 a 9 anos        476   468   460   452   444   437   
                          10 a 19 anos      565   555   545   534   524   515   
Tunas do Paraná Masculino 70 a 79 anos       79    82    85    88    91    95   
                          80 anos ou mais    25    27    28    30    31    33   

                                           2020  2021  2022  
Município       Sexo      Faixa Etária                       
Adrianópolis    Feminino  0 a 9 anos        429   422   415  
                          10 a 19 anos      505   496   487  
Tunas do Paraná Masculino 70 a 79 anos       98   102   106  
                          80 anos ou mais    35    37    39  

[4 rows x 23 columns]

In [20]:
df = df_pop[[2022]]
df = df.rename(columns={2022:'População 2022'})
df.iloc[np.r_[0:2, -2:0]]

População 2022
Município       Sexo      Faixa Etária                   
Adrianópolis    Feminino  0 a 9 anos                  415
                          10 a 19 anos                487
Tunas do Paraná Masculino 70 a 79 anos                106
                          80 anos ou mais              39

## Nascimentos

In [21]:
df_birth_rate = pd.read_csv(
    '../../br_demography/results/tab/taxa_natalidade_projetada_rmc_2022_2045_2.csv'
    , decimal=','
    , sep=';'
    , index_col=['Município', 'Faixa Etária']
    , encoding='utf=-16'
    , dtype={'Faixa Etária':categories}
    )
df_birth_rate = df_birth_rate.iloc[:,3:]
df_birth_rate.columns = df_birth_rate.columns.astype(int)
df_birth_rate.sort_index(inplace=True)
df_birth_rate.iloc[np.r_[0:2, -2:0]]

2020       2021       2022       2023  \
Município       Faixa Etária                                               
Adrianópolis    10 a 19 anos  30.951423  30.832987  30.715006  30.597475   
                20 a 29 anos  85.068836  84.807296  84.546561  84.286627   
Tunas do Paraná 30 a 39 anos  60.243064  59.097119  57.972972  56.870209   
                40 a 49 anos   5.537862   5.317346   5.105610   4.902307   

                                   2024       2025       2026       2027  \
Município       Faixa Etária                                               
Adrianópolis    10 a 19 anos  30.480394  30.363762  30.247575  30.131833   
                20 a 29 anos  84.027492  83.769154  83.511610  83.254858   
Tunas do Paraná 30 a 39 anos  55.788422  54.727214  53.686191  52.664971   
                40 a 49 anos   4.707098   4.519663   4.339691   4.166886   

                                   2028       2029  ...       2036       2037  \
Município       Faixa Etária                        ...                         
Adrianópolis    10 a 19 anos  30.016534  29.901677  ...  29.109885  28.998496   
                20 a 29 anos  82.998895  82.743720  ...  80.979322  80.730356   
Tunas do Paraná 30 a 39 anos  51.663177  50.680439  ...  44.305247  43.462472   
                40 a 49 anos   4.000962   3.841645  ...   2.890591   2.775489   

                                   2038       2039       2040       2041  \
Município       Faixa Etária                                               
Adrianópolis    10 a 19 anos  28.887534  28.776996  28.666882  28.557188   
                20 a 29 anos  80.482154  80.234716  79.988039  79.742120   
Tunas do Paraná 30 a 39 anos  42.635727  41.824709  41.029119  40.248662   
                40 a 49 anos   2.664970   2.558851   2.456959   2.359123   

                                   2042       2043       2044       2045  
Município       Faixa Etária                                              
Adrianópolis    10 a 19 anos  28.447915  28.339059  28.230620  28.122596  
                20 a 29 anos  79.496957  79.252548  79.008890  78.765982  
Tunas do Paraná 30 a 39 anos  39.483051  38.732003  37.995242  37.272496  
                40 a 49 anos   2.265184   2.174985   2.088378   2.005219  

[4 rows x 26 columns]

In [22]:
df_nascimentos_2022_2031 = ((
    df_pop.reset_index()
    .loc[df_pop.reset_index()['Sexo'] == 'Feminino'].drop(columns=['Sexo'])
    .set_index(['Município', 'Faixa Etária'])
    .loc[:,2022]
).mul(
    (
    df_birth_rate
    .loc[:,list(range(2022, 2032))]
    .mean(axis=1)
    .to_frame()
    .div(1000)
    .mul(10)
    .squeeze()
    )
)
    .to_frame()
    .reset_index()
    .groupby(by=['Município'])
    .sum()
    .rename(columns={0:'Nascimentos até 2022-2031'})
)

In [23]:
df_nascimentos_2022_2031

,Nascimentos até 2022-2031
Município,
Adrianópolis,692.508228
Agudos do Sul,1182.222987
Almirante Tamandaré,17206.166301
Araucária,23592.601070
Balsa Nova,2203.447179
Bocaiúva do Sul,1610.004978
Campina Grande do Sul,7922.267994
Campo Largo,16768.594301
Campo Magro,4175.274362


In [24]:
df.reset_index()[df.reset_index()['Faixa Etária'] == '0 a 9 anos']

,Município,Sexo,Faixa Etária,População 2022
0,Adrianópolis,Feminino,0 a 9 anos,415
9,Adrianópolis,Masculino,0 a 9 anos,406
18,Agudos do Sul,Feminino,0 a 9 anos,694
27,Agudos do Sul,Masculino,0 a 9 anos,753
36,Almirante Tamandaré,Feminino,0 a 9 anos,8788
45,Almirante Tamandaré,Masculino,0 a 9 anos,9268
54,Araucária,Feminino,0 a 9 anos,10457
63,Araucária,Masculino,0 a 9 anos,10931
72,Balsa Nova,Feminino,0 a 9 anos,866
81,Balsa Nova,Masculino,0 a 9 anos,940


In [25]:
df_nascimentos_2022_2031['Masculino'] = df_nascimentos_2022_2031 * 0.51
df_nascimentos_2022_2031['Feminino'] = df_nascimentos_2022_2031['Nascimentos até 2022-2031'] - df_nascimentos_2022_2031['Masculino']


In [26]:
df['Nascimentos 2022-2031'] = (
    df_nascimentos_2022_2031
    .reset_index()
    .loc[:,['Município', 'Masculino', 'Feminino']]
    .melt(id_vars=['Município'], var_name='Sexo', value_name='Nascimentos 2022-2031')
    .assign(Faixa='0 a 9 anos')
    .rename(columns={'Faixa':'Faixa Etária'})
    .astype({'Faixa Etária': categories})
    .set_index(['Município', 'Sexo', 'Faixa Etária'])
    .astype(int)
    .sort_index()
).squeeze()
df = df.fillna(0)
df = df.astype(int)
df.head(2)

População 2022  Nascimentos 2022-2031
Município    Sexo     Faixa Etária                                       
Adrianópolis Feminino 0 a 9 anos               415                    339
                      10 a 19 anos             487                      0

In [27]:
df.loc[df.index.get_level_values('Faixa Etária') == '0 a 9 anos']

População 2022  \
Município             Sexo      Faixa Etária                   
Adrianópolis          Feminino  0 a 9 anos               415   
                      Masculino 0 a 9 anos               406   
Agudos do Sul         Feminino  0 a 9 anos               694   
                      Masculino 0 a 9 anos               753   
Almirante Tamandaré   Feminino  0 a 9 anos              8788   
                      Masculino 0 a 9 anos              9268   
Araucária             Feminino  0 a 9 anos             10457   
                      Masculino 0 a 9 anos             10931   
Balsa Nova            Feminino  0 a 9 anos               866   
                      Masculino 0 a 9 anos               940   
Bocaiúva do Sul       Feminino  0 a 9 anos               990   
                      Masculino 0 a 9 anos               997   
Campina Grande do Sul Feminino  0 a 9 anos              3342   
                      Masculino 0 a 9 anos              3388   
Campo Largo           Feminino  0 a 9 anos              8580   
                      Masculino 0 a 9 anos              8855   
Campo Magro           Feminino  0 a 9 anos              1956   
                      Masculino 0 a 9 anos              2006   
Campo do Tenente      Feminino  0 a 9 anos               588   
                      Masculino 0 a 9 anos               590   
Cerro Azul            Feminino  0 a 9 anos              1061   
                      Masculino 0 a 9 anos              1194   
Colombo               Feminino  0 a 9 anos             15585   
                      Masculino 0 a 9 anos             15965   
Contenda              Feminino  0 a 9 anos              1301   
                      Masculino 0 a 9 anos              1312   
Curitiba              Feminino  0 a 9 anos             90477   
                      Masculino 0 a 9 anos             93722   
Doutor Ulysses        Feminino  0 a 9 anos               367   
                      Masculino 0 a 9 anos               404   
Fazenda Rio Grande    Feminino  0 a 9 anos             11472   
                      Masculino 0 a 9 anos             11938   
Itaperuçu             Feminino  0 a 9 anos              2426   
                      Masculino 0 a 9 anos              2478   
Lapa                  Feminino  0 a 9 anos              2696   
                      Masculino 0 a 9 anos              2696   
Mandirituba           Feminino  0 a 9 anos              1793   
                      Masculino 0 a 9 anos              1912   
Pinhais               Feminino  0 a 9 anos              7413   
                      Masculino 0 a 9 anos              7738   
Piraquara             Feminino  0 a 9 anos              8260   
                      Masculino 0 a 9 anos              8764   
Piên                  Feminino  0 a 9 anos              1026   
                      Masculino 0 a 9 anos               981   
Quatro Barras         Feminino  0 a 9 anos              1511   
                      Masculino 0 a 9 anos              1654   
Quitandinha           Feminino  0 a 9 anos              1109   
                      Masculino 0 a 9 anos              1229   
Rio Branco do Sul     Feminino  0 a 9 anos              2615   
                      Masculino 0 a 9 anos              2793   
Rio Negro             Feminino  0 a 9 anos              1977   
                      Masculino 0 a 9 anos              2031   
São José dos Pinhais  Feminino  0 a 9 anos             22014   
                      Masculino 0 a 9 anos             22896   
Tijucas do Sul        Feminino  0 a 9 anos              1203   
                      Masculino 0 a 9 anos              1267   
Tunas do Paraná       Feminino  0 a 9 anos               528   
                      Masculino 0 a 9 anos               541   

                                              Nascimentos 2022-2031  
Município             Sexo      Faixa Etária                         
Adrianópolis          Feminino  0 a 9 anos                      3

## Taxa de Sobrevivência

In [28]:
df_survival_rate = pd.read_csv(
    '../../br_demography/results/tab/taxa_sobrevivencia_projetada_pdui_2022_2045.csv'
    , decimal=','
    , sep=';'
    , index_col=['Município', 'Sexo', 'Faixa Etária']
    , encoding='utf=-16'
    , dtype={'Faixa Etária':categories}
    )
df_survival_rate = df_survival_rate.iloc[:,3:]
df_survival_rate.columns = df_survival_rate.columns.astype(int)
df_survival_rate.sort_index(inplace=True)
df_survival_rate.head(2)

2022      2023      2024      2025  \
Município    Sexo     Faixa Etária                                           
Adrianópolis Feminino 0 a 9 anos    0.997499  0.997451  0.997403  0.997354   
                      10 a 19 anos  0.995674  0.995518  0.995361  0.995204   

                                        2026      2027      2028      2029  \
Município    Sexo     Faixa Etária                                           
Adrianópolis Feminino 0 a 9 anos    0.997306  0.997258  0.997209  0.997161   
                      10 a 19 anos  0.995047  0.994891  0.994734  0.994577   

                                        2030      2031  ...      2036  \
Município    Sexo     Faixa Etária                      ...             
Adrianópolis Feminino 0 a 9 anos    0.997112  0.997064  ...  0.996822   
                      10 a 19 anos  0.994421  0.994264  ...  0.993481   

                                        2037      2038      2039      2040  \
Município    Sexo     Faixa Etária                                           
Adrianópolis Feminino 0 a 9 anos    0.996774  0.996725  0.996677  0.996629   
                      10 a 19 anos  0.993324  0.993167  0.993010  0.992854   

                                        2041      2042      2043      2044  \
Município    Sexo     Faixa Etária                                           
Adrianópolis Feminino 0 a 9 anos    0.996580  0.996532  0.996483  0.996435   
                      10 a 19 anos  0.992697  0.992540  0.992384  0.992227   

                                        2045  
Município    Sexo     Faixa Etária            
Adrianópolis Feminino 0 a 9 anos    0.996387  
                      10 a 19 anos  0.992070  

[2 rows x 24 columns]

In [29]:
df['Taxa de Sobrevivência 2022-2031'] = df_survival_rate.loc[:,list(range(2022, 2032))].prod(axis=1).to_frame().rename(columns={0:'2032'}).squeeze()

In [30]:
faixas_etarias_anteriores = ['0 a 9 anos', '10 a 19 anos', '20 a 29 anos', '30 a 39 anos', '40 a 49 anos', '50 a 59 anos', '60 a 69 anos', '70 a 79 anos', '80 anos ou mais']
faixas_etarias_posteriores = faixas_etarias_anteriores.copy()

faixas_etarias_posteriores.append(faixas_etarias_anteriores[-1])
faixas_etarias_posteriores = faixas_etarias_posteriores[1:]
dict_faixas_compat = {faixa_anterior:faixa_posterior for faixa_anterior, faixa_posterior in zip(faixas_etarias_anteriores, faixas_etarias_posteriores)}
dict_faixas_compat

{'0 a 9 anos': '10 a 19 anos',
 '10 a 19 anos': '20 a 29 anos',
 '20 a 29 anos': '30 a 39 anos',
 '30 a 39 anos': '40 a 49 anos',
 '40 a 49 anos': '50 a 59 anos',
 '50 a 59 anos': '60 a 69 anos',
 '60 a 69 anos': '70 a 79 anos',
 '70 a 79 anos': '80 anos ou mais',
 '80 anos ou mais': '80 anos ou mais'}

In [31]:

df = df.join(
    (
    df
    .assign(Próxima_Faixa_Etária=df.reset_index()['Faixa Etária'].map(dict_faixas_compat).values)
    .reset_index()
    .set_index(['Município', 'Sexo', 'Faixa Etária', 'Próxima_Faixa_Etária'])
    .loc[:,'População 2022']
    .mul(df['Taxa de Sobrevivência 2022-2031'])
    .reset_index()
    .rename(columns={0:'Sobrevivência 2022-2031'})
    .astype({'Sobrevivência 2022-2031':int, 'Próxima_Faixa_Etária':categories})
    .drop(columns=['Faixa Etária'])
    .rename(columns={'Próxima_Faixa_Etária':'Faixa Etária'})
    .groupby(by=['Município', 'Sexo', 'Faixa Etária'])
    .sum()
#    .set_index(['Município', 'Sexo', 'Faixa Etária'])
    )
)

In [32]:
df['População 2032'] = df['Nascimentos 2022-2031'] + df['Sobrevivência 2022-2031']
df_pop[2032] = df['População 2032']

In [33]:
df

População 2022  \
Município       Sexo      Faixa Etária                      
Adrianópolis    Feminino  0 a 9 anos                  415   
                          10 a 19 anos                487   
                          20 a 29 anos                384   
                          30 a 39 anos                400   
                          40 a 49 anos                402   
...                                                   ...   
Tunas do Paraná Masculino 40 a 49 anos                416   
                          50 a 59 anos                306   
                          60 a 69 anos                225   
                          70 a 79 anos                106   
                          80 anos ou mais              39   

                                           Nascimentos 2022-2031  \
Município       Sexo      Faixa Etária                             
Adrianópolis    Feminino  0 a 9 anos                         339   
                          10 a 19 anos                         0   
                          20 a 29 anos                         0   
                          30 a 39 anos                         0   
                          40 a 49 anos                         0   
...                                                          ...   
Tunas do Paraná Masculino 40 a 49 anos                         0   
                          50 a 59 anos                         0   
                          60 a 69 anos                         0   
                          70 a 79 anos                         0   
                          80 anos ou mais                      0   

                                           Taxa de Sobrevivência 2022-2031  \
Município       Sexo      Faixa Etária                                       
Adrianópolis    Feminino  0 a 9 anos                              0.973147   
                          10 a 19 anos                            0.950814   
                          20 a 29 anos                            0.974092   
                          30 a 39 anos                            0.975056   
                          40 a 49 anos                            0.969484   
...                                                                    ...   
Tunas do Paraná Masculino 40 a 49 anos                            0.986926   
                          50 a 59 anos                            0.896420   
                          60 a 69 anos                            0.725113   
                          70 a 79 anos                            0.705727   
                          80 anos ou mais                         0.304209   

                                           Sobrevivência 2022-2031  \
Município       Sexo      Faixa Etária                               
Adrianópolis    Feminino  0 a 9 anos                             0   
                          10 a 19 anos                         403   
                          20 a 29 anos                         463   
                          30 a 39 anos                         374   
                          40 a 49 anos                         390   
...                                                            ...   
Tunas do Paraná Masculino 40 a 49 anos                         465   
                          50 a 59 anos                         410   
                          60 a 69 anos                         274   
                          70 a 79 anos                         163   
                          80 anos ou mais                       85   

                                           População 2032  
Município       Sexo      Faixa Etária                     
Adrianópolis    Feminino  0 a 9 anos                  339  
                          10 a 19 anos                403  
                          20 a 29 anos                463  
                          30 a 39 anos                374  
                          40 a 49 anos                390  
...               

# Projeção para 2042

## População do Ano Inicial

## Nascimentos

In [34]:
df['População 2032']

Município        Sexo       Faixa Etária   
Adrianópolis     Feminino   0 a 9 anos         339
                            10 a 19 anos       403
                            20 a 29 anos       463
                            30 a 39 anos       374
                            40 a 49 anos       390
                                              ... 
Tunas do Paraná  Masculino  40 a 49 anos       465
                            50 a 59 anos       410
                            60 a 69 anos       274
                            70 a 79 anos       163
                            80 anos ou mais     85
Name: População 2032, Length: 522, dtype: int64

In [35]:
df_nascimentos_2032_2041 = (
    (
    df_pop.reset_index()
    .loc[df_pop.reset_index()['Sexo'] == 'Feminino'].drop(columns=['Sexo'])
    .set_index(['Município', 'Faixa Etária'])
    .loc[:,2032]
).mul(
    (
    df_birth_rate
    .loc[:,list(range(2032, 2042))]
    .mean(axis=1)
    .to_frame()
    .div(1000)
    .mul(10)
    .squeeze()
    )
)
    .to_frame()
    .reset_index()
    .groupby(by=['Município'])
    .sum()
    .rename(columns={0:'Nascimentos 2032-2041'})
)

In [36]:
df_nascimentos_2032_2041['Masculino'] = df_nascimentos_2032_2041 * 0.51

df_nascimentos_2032_2041['Feminino'] = df_nascimentos_2032_2041['Nascimentos 2032-2041'] - df_nascimentos_2032_2041['Masculino']


In [37]:
df['Nascimentos 2032-2041'] = (
    df_nascimentos_2032_2041
    .reset_index()
    .loc[:,['Município', 'Masculino', 'Feminino']]
    .melt(id_vars=['Município'], var_name='Sexo', value_name='Nascimentos 2032-2041')
    .assign(Faixa='0 a 9 anos')
    .rename(columns={'Faixa':'Faixa Etária'})
    .astype({'Faixa Etária': categories})
    .set_index(['Município', 'Sexo', 'Faixa Etária'])
    .astype(int)
    .sort_index()
).squeeze()
df = df.fillna(0)
df = df.astype(int)
df.head(2)

População 2022  Nascimentos 2022-2031  \
Município    Sexo     Faixa Etária                                          
Adrianópolis Feminino 0 a 9 anos               415                    339   
                      10 a 19 anos             487                      0   

                                    Taxa de Sobrevivência 2022-2031  \
Município    Sexo     Faixa Etária                                    
Adrianópolis Feminino 0 a 9 anos                                  0   
                      10 a 19 anos                                0   

                                    Sobrevivência 2022-2031  População 2032  \
Município    Sexo     Faixa Etária                                            
Adrianópolis Feminino 0 a 9 anos                          0             339   
                      10 a 19 anos                      403             403   

                                    Nascimentos 2032-2041  
Município    Sexo     Faixa Etária                         
Adrianópolis Feminino 0 a 9 anos                      338  
                      10 a 19 anos                      0

In [38]:
df.loc[df.index.get_level_values('Faixa Etária') == '0 a 9 anos'][['Nascimentos 2022-2031', 'Nascimentos 2032-2041']]

Nascimentos 2022-2031  \
Município             Sexo      Faixa Etária                          
Adrianópolis          Feminino  0 a 9 anos                      339   
                      Masculino 0 a 9 anos                      353   
Agudos do Sul         Feminino  0 a 9 anos                      579   
                      Masculino 0 a 9 anos                      602   
Almirante Tamandaré   Feminino  0 a 9 anos                     8431   
                      Masculino 0 a 9 anos                     8775   
Araucária             Feminino  0 a 9 anos                    11560   
                      Masculino 0 a 9 anos                    12032   
Balsa Nova            Feminino  0 a 9 anos                     1079   
                      Masculino 0 a 9 anos                     1123   
Bocaiúva do Sul       Feminino  0 a 9 anos                      788   
                      Masculino 0 a 9 anos                      821   
Campina Grande do Sul Feminino  0 a 9 anos                     3881   
                      Masculino 0 a 9 anos                     4040   
Campo Largo           Feminino  0 a 9 anos                     8216   
                      Masculino 0 a 9 anos                     8551   
Campo Magro           Feminino  0 a 9 anos                     2045   
                      Masculino 0 a 9 anos                     2129   
Campo do Tenente      Feminino  0 a 9 anos                      548   
                      Masculino 0 a 9 anos                      571   
Cerro Azul            Feminino  0 a 9 anos                      809   
                      Masculino 0 a 9 anos                      842   
Colombo               Feminino  0 a 9 anos                    16164   
                      Masculino 0 a 9 anos                    16824   
Contenda              Feminino  0 a 9 anos                     1359   
                      Masculino 0 a 9 anos                     1415   
Curitiba              Feminino  0 a 9 anos                    98121   
                      Masculino 0 a 9 anos                   102126   
Doutor Ulysses        Feminino  0 a 9 anos                      246   
                      Masculino 0 a 9 anos                      256   
Fazenda Rio Grande    Feminino  0 a 9 anos                    11937   
                      Masculino 0 a 9 anos                    12424   
Itaperuçu             Feminino  0 a 9 anos                     2386   
                      Masculino 0 a 9 anos                     2484   
Lapa                  Feminino  0 a 9 anos                     2466   
                      Masculino 0 a 9 anos                     2566   
Mandirituba           Feminino  0 a 9 anos                     1558   
                      Masculino 0 a 9 anos                     1622   
Pinhais               Feminino  0 a 9 anos                     8554   
                      Masculino 0 a 9 anos                     8903   
Piraquara             Feminino  0 a 9 anos                     7686   
                      Masculino 0 a 9 anos                     8000   
Piên                  Feminino  0 a 9 anos                      819   
                      Masculino 0 a 9 anos                      853   
Quatro Barras         Feminino  0 a 9 anos                     1601   
                      Masculino 0 a 9 anos                     1667   
Quitandinha           Feminino  0 a 9 anos                      985   
                      Masculino 0 a 9 anos                     1025   
Rio Branco do Sul     Feminino  0 a 9 anos                     2272   
                      Masculino 0 a 9 anos                     2365   
Rio Negro             Feminino  0 a 9 anos                     1413   
                      Masculino 0 a 9 anos                     1470   
São José dos Pinhais  Feminino  0 a 9 anos                    22794   
                      Masculino 0 a 9 anos                    23724   
Tijucas do Sul        Feminino  0 a 9 anos                     1117   

## Taxa de Sobrevivência

In [39]:
df['Taxa de Sobrevivência 2032-2041'] = df_survival_rate.loc[:,list(range(2032, 2042))].prod(axis=1).to_frame().rename(columns={0:'2042'}).squeeze()

In [40]:
faixas_etarias_anteriores = ['0 a 9 anos', '10 a 19 anos', '20 a 29 anos', '30 a 39 anos', '40 a 49 anos', '50 a 59 anos', '60 a 69 anos', '70 a 79 anos', '80 anos ou mais']
faixas_etarias_posteriores = faixas_etarias_anteriores.copy()

faixas_etarias_posteriores.append(faixas_etarias_anteriores[-1])
faixas_etarias_posteriores = faixas_etarias_posteriores[1:]
dict_faixas_compat = {faixa_anterior:faixa_posterior for faixa_anterior, faixa_posterior in zip(faixas_etarias_anteriores, faixas_etarias_posteriores)}
dict_faixas_compat

{'0 a 9 anos': '10 a 19 anos',
 '10 a 19 anos': '20 a 29 anos',
 '20 a 29 anos': '30 a 39 anos',
 '30 a 39 anos': '40 a 49 anos',
 '40 a 49 anos': '50 a 59 anos',
 '50 a 59 anos': '60 a 69 anos',
 '60 a 69 anos': '70 a 79 anos',
 '70 a 79 anos': '80 anos ou mais',
 '80 anos ou mais': '80 anos ou mais'}

In [41]:

df = df.join(
    (
    df
    .assign(Próxima_Faixa_Etária=df.reset_index()['Faixa Etária'].map(dict_faixas_compat).values)
    .reset_index()
    .set_index(['Município', 'Sexo', 'Faixa Etária', 'Próxima_Faixa_Etária'])
    .loc[:,'População 2032']
    .mul(df['Taxa de Sobrevivência 2032-2041'])
    .reset_index()
    .rename(columns={0:'Sobrevivência 2032-2041'})
    .astype({'Sobrevivência 2032-2041':int, 'Próxima_Faixa_Etária':categories})
    .drop(columns=['Faixa Etária'])
    .rename(columns={'Próxima_Faixa_Etária':'Faixa Etária'})
    .groupby(by=['Município', 'Sexo', 'Faixa Etária'])
    .sum()
#    .set_index(['Município', 'Sexo', 'Faixa Etária'])
    )
)

In [42]:
df['População 2042'] = df['Nascimentos 2032-2041'] + df['Sobrevivência 2032-2041']

In [43]:
df[['População 2032', 'População 2042']].sum()

População 2032    3834020
População 2042    3980118
dtype: int64

In [44]:
df

População 2022  \
Município       Sexo      Faixa Etária                      
Adrianópolis    Feminino  0 a 9 anos                  415   
                          10 a 19 anos                487   
                          20 a 29 anos                384   
                          30 a 39 anos                400   
                          40 a 49 anos                402   
...                                                   ...   
Tunas do Paraná Masculino 40 a 49 anos                416   
                          50 a 59 anos                306   
                          60 a 69 anos                225   
                          70 a 79 anos                106   
                          80 anos ou mais              39   

                                           Nascimentos 2022-2031  \
Município       Sexo      Faixa Etária                             
Adrianópolis    Feminino  0 a 9 anos                         339   
                          10 a 19 anos                         0   
                          20 a 29 anos                         0   
                          30 a 39 anos                         0   
                          40 a 49 anos                         0   
...                                                          ...   
Tunas do Paraná Masculino 40 a 49 anos                         0   
                          50 a 59 anos                         0   
                          60 a 69 anos                         0   
                          70 a 79 anos                         0   
                          80 anos ou mais                      0   

                                           Taxa de Sobrevivência 2022-2031  \
Município       Sexo      Faixa Etária                                       
Adrianópolis    Feminino  0 a 9 anos                                     0   
                          10 a 19 anos                                   0   
                          20 a 29 anos                                   0   
                          30 a 39 anos                                   0   
                          40 a 49 anos                                   0   
...                                                                    ...   
Tunas do Paraná Masculino 40 a 49 anos                                   0   
                          50 a 59 anos                                   0   
                          60 a 69 anos                                   0   
                          70 a 79 anos                                   0   
                          80 anos ou mais                                0   

                                           Sobrevivência 2022-2031  \
Município       Sexo      Faixa Etária                               
Adrianópolis    Feminino  0 a 9 anos                             0   
                          10 a 19 anos                         403   
                          20 a 29 anos                         463   
                          30 a 39 anos                         374   
                          40 a 49 anos                         390   
...                                                            ...   
Tunas do Paraná Masculino 40 a 49 anos                         465   
                          50 a 59 anos                         410   
                          60 a 69 anos                         274   
                          70 a 79 anos                         163   
                          80 anos ou mais                       85   

                                           População 2032  \
Município       Sexo      Faixa Etária                      
Adrianópolis    Feminino  0 a 9 anos                  339   
                          10 a 19 anos                403   
                          20 a 29 anos                463   
                          30 a 39 anos                374   
                          40 a 49 anos                390   
...        

In [45]:
df.to_csv(path_or_buf='../../br_demography/results/tab/projecao_populacional_rmc_2022_2042_2.csv', sep=';', decimal=',', encoding='utf-16')